In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout , BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from tensorflow.keras.callbacks import ReduceLROnPlateau
import cv2 as cv
import h5py

# Load Data

In [2]:
labels = ['call_me','fingers_crossed','up','okay','paper','rock','rock_on','scissor','peace','thumbs']
x_signs = []
y_signs = []
path = "images"
for i in os.listdir(path):
    for j in os.listdir(path + "/" + i):
        img =  cv.imread(path + "/" + i + '/' + j,0)
        img =  cv.resize(img,(64,64),interpolation = cv.INTER_AREA)
        img = np.array(img)
        x_signs.append(img)
        y_signs.append(labels.index(i))

In [3]:
x = np.array(x_signs)
x.shape

(5244, 64, 64)

In [4]:
X = np.array(x_signs)
Y = np.array(y_signs)
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, Y, train_size=0.75, random_state = 42)

In [5]:
x_train.shape

(3933, 64, 64)

In [6]:
y_train.shape

(3933,)

In [7]:
x_train[11].shape

(64, 64)

# Thresholding

In [8]:
train_x = []
train_y = []
for i in range(len(x_train)):
    _, bw_image = cv.threshold(x_train[i], 120, 255, cv.THRESH_BINARY)
    y = y_train[i]
    # orginal image
    train_x.append(bw_image)
    train_y.append(y)

    #rotate 100 degree
    train_x.append(cv.flip(bw_image,1))
    train_y.append(y)
train_x  = np.array(train_x) 
train_y  = np.array(train_y)

In [9]:
test_x = []
test_y = []
for i in range(len(x_test)):
    _, bw_image = cv.threshold(x_test[i], 120, 255, cv.THRESH_BINARY)
    y = y_test[i]
    # orginal image
    test_x.append(bw_image)
    test_y.append(y)
    #rotate 90 degree
    test_x.append(cv.flip(bw_image,1))
    test_y.append(y)
test_x = np.array(test_x) 
test_y = np.array(test_y) 

## Label Binarizer

In [10]:
from sklearn.preprocessing import LabelBinarizer

In [11]:
label_binarizer = LabelBinarizer()

In [12]:
y_train = label_binarizer.fit_transform(train_y)
y_test = label_binarizer.fit_transform(test_y)

In [13]:
y_train[:7]

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]])

# Normalization

In [14]:
x_train = train_x.reshape(-1,64,64,1)
x_test = test_x.reshape(-1,64,64,1)

In [15]:
x_train.shape

(7866, 64, 64, 1)

# Data Augmentations

In [16]:
data_generator = ImageDataGenerator(
    rotation_range = 0.1,
    zoom_range = 0.1,
    width_shift_range=0.1,
    height_shift_range=0.1
)

In [17]:
data_generator.fit(x_train)

# Build Model

In [18]:
len(labels)

10

In [19]:
model = Sequential()
#first layer
model.add(Conv2D(75,(3,3),strides=1,padding='same',activation='relu',input_shape = (64,64,1)))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2),strides=2, padding='same'))
#second layer
model.add(Conv2D(50,(3,3),strides=1,padding='same',activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2),strides=2, padding='same'))
#third layer
model.add(Conv2D(25,(3,3),strides=1,padding='same',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2),strides=2, padding='same'))
#flatten
model.add(Flatten())
#First fully connected layer
model.add(Dense(units=512,activation='relu'))
model.add(Dropout(0.3))
#Second fully connected layer
model.add(Dense(units=10,activation='softmax'))

# Compile Model

In [20]:
tensorflow.debugging.set_log_device_placement(True)

In [21]:
model.compile(optimizer='adam',loss = 'categorical_crossentropy', metrics=['accuracy'])

# Set learning rate


In [22]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5,verbose=1,patience=2, min_lr=0.00001)

# Train the Model

In [23]:
model.fit(data_generator.flow(x_train,y_train,batch_size = 128),epochs=30,validation_data=(x_test,y_test),callbacks=[learning_rate_reduction])

Epoch 1/30
62/62 [==============================] - 4s 25ms/step - loss: 1.1630 - accuracy: 0.6468 - val_loss: 3.5966 - val_accuracy: 0.3707
Epoch 2/30
62/62 [==============================] - 1s 20ms/step - loss: 0.2953 - accuracy: 0.9069 - val_loss: 1.2154 - val_accuracy: 0.6335
Epoch 3/30
62/62 [==============================] - 1s 21ms/step - loss: 0.1569 - accuracy: 0.9522 - val_loss: 0.0977 - val_accuracy: 0.9653
Epoch 4/30
62/62 [==============================] - 1s 20ms/step - loss: 0.1205 - accuracy: 0.9647 - val_loss: 0.0532 - val_accuracy: 0.9847
Epoch 5/30
62/62 [==============================] - 1s 20ms/step - loss: 0.0854 - accuracy: 0.9725 - val_loss: 0.0555 - val_accuracy: 0.9828
Epoch 6/30
62/62 [==============================] - 1s 21ms/step - loss: 0.0669 - accuracy: 0.9807 - val_loss: 0.0730 - val_accuracy: 0.9752

Epoch 00006: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
Epoch 7/30
62/62 [==============================] - 1s 21ms/step - loss: 

In [24]:
model.evaluate(x_test,y_test)

82/82 [==============================] - 0s 1ms/step - loss: 0.0239 - accuracy: 0.9928


[0.023875663056969643, 0.9927536249160767]

# Save Model

In [25]:
model_json = model.to_json()
with open("model-bw10.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights('model-bw10.h5')